In [ ]:
!pip install gradio > /dev/null

In [ ]:
! python -m spacy download en 
! pip install wordcloud > /dev/null
! wget https://gitlab.com/andras.simonyi/10_days_AI_training_data/raw/master/sentiment.tsv?inline=false -O sentiment.tsv

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/alatus/anaconda3/envs/10_days_AI/lib/python3.7/site-packages/en_core_web_sm
-->
/home/alatus/anaconda3/envs/10_days_AI/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
--2020-10-27 17:27:09--  https://gitlab.com/andras.simonyi/10_days_AI_training_data/raw/master/sentiment.tsv?inline=false
Resolving gitlab.com (gitlab.com)... 172.65.251.78
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘sentiment.tsv’

sentiment.tsv           [ <=>                ] 437.05K  2.80MB/s    in 0.2s    

2020-10-27 17:27:10 (2.80 MB/s) - ‘sentiment.tsv’ saved [447540]



# Task: sentiment classification

The task is to classify one-sentence long movie reviews/opinions according to the sentiment they express. There are only two categories: positive and negative sentiment.


> "Data source: [UMICH SI650 - Sentiment Classification](https://www.kaggle.com/c/si650winter11/data)

> Training data: 7086 lines. 
  
> Format: 1|0 (tab) sentence

> Test data: 33052 lines, each contains one sentence. 

> The data was originally collected from opinmind.com (which is no longer active)."

The data is in the file "sentiment.tsv".

# Loading the data

In [ ]:
import pandas as pd

df = pd.read_csv('sentiment.tsv', sep='\t', 
                 quoting=3, # Quotes are _never_ field separators
                 header=None)

df.head()

df = df[[1,0]] # reorder columns

df.rename(columns={1:"text", 0:"sentiment"}, inplace=True) # rename columns

df.head()

,text,sentiment
0,The Da Vinci Code book is just awesome.,1
1,this was the first clive cussler i've ever rea...,1
2,i liked the Da Vinci Code a lot.,1
3,i liked the Da Vinci Code a lot.,1
4,I liked the Da Vinci Code but it ultimatly did...,1


# Splitting into train, validation and test

Before doing anything else (!) we divide our data into train, validation and test parts,

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test_valid = train_test_split(df, test_size = 0.2, shuffle=True, 
                                           random_state=13) # fix the seed

df_test, df_valid = train_test_split(df_test_valid, test_size = 0.5)

print(len(df_train), len(df_valid), len(df_test))

5668 709 709


# Inspecting the data

In [ ]:
df_train.describe()

,sentiment
count,5668.000000
mean,0.559104
std,0.496538
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


We can examine the lengths of sentences as well.

In [ ]:
n_chars = df_train.text.apply(lambda x: len(x))

n_chars.describe()

count    5668.000000
mean       60.100565
std        37.931478
min        18.000000
25%        32.000000
50%        48.000000
75%        77.000000
max       203.000000
Name: text, dtype: float64

The first sentence with the maximal length:

In [ ]:
long_sentence = df_train.loc[n_chars.idxmax(), "text"]
long_sentence

'A mother in Georgia wants her local school board to take Harry Potter out of the schools and libraries because, in her opinion, reading Harry Potter leads to witchcraft, which according to her is evil...'

# Bag of words (BoW) representation of the texts

We will represent each text as a (sparse) vector of lemma (word root) counts for frequent lemmas in the training data. 

For tokenization and lemmatization we use [spaCy](https://spacy.io/), an open source Python NLP library, which can produce a list of unique lemma ids from the text.

In [ ]:
import spacy

nlp = spacy.load("en", disable=["parser", "ner"]) # We need only the tokenizer

spaCy can produce spaCy Doc objects from texts that contain their linguistic analysis, among others lemmas and their unique spaCy string ids.

In [ ]:
doc = nlp(long_sentence)
type(doc)

print([token.lemma_ for token in doc ]) # Lemmas

print([token.lemma for token in doc]) # Corresponding unique ids

['a', 'mother', 'in', 'Georgia', 'want', '-PRON-', 'local', 'school', 'board', 'to', 'take', 'Harry', 'Potter', 'out', 'of', 'the', 'school', 'and', 'library', 'because', ',', 'in', '-PRON-', 'opinion', ',', 'read', 'Harry', 'Potter', 'lead', 'to', 'witchcraft', ',', 'which', 'accord', 'to', '-PRON-', 'be', 'evil', '...']
[11901859001352538922, 7963322251145911254, 3002984154512732771, 309210702643012516, 7597692042947428029, 561228191312463089, 16319852998319793599, 13293160603192985325, 14899812206273857344, 3791531372978436496, 6789454535283781228, 5164779919001708464, 2416965663249996073, 1696981056005371314, 886050111519832510, 7425985699627899538, 13293160603192985325, 2283656566040971221, 1785747669126016609, 16950148841647037698, 2593208677638477497, 3002984154512732771, 561228191312463089, 14536103007527724270, 2593208677638477497, 11792590063656742891, 5164779919001708464, 2416965663249996073, 82546335403996757, 3791531372978436496, 17905374590688478165, 2593208677638477497, 

Now we have to convert these lists into BoW vectors. We could "roll our own", but, fortunately, scikit-learn has a feature extractor doing exactly that, the [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) so, for the sake of simplicity, we will use that along with spaCy.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(analyzer=lambda s: [token.lemma for token in nlp(s)], # use spaCy for analysis
                     min_df= 0.001) # Ignore lemmas with lower document frequency
cv

CountVectorizer(analyzer=<function <lambda> at 0x7ffb08547b00>, min_df=0.001)

In [ ]:
sents = ["I hate this movie.", "The movie is the worst I've seen."]
bows = cv.fit_transform(sents).toarray() # CountVectorizer produces a sparse matrix so we convert to ndarray
bows

array([[1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 2, 0, 1, 1, 1, 1, 1, 1]])

Using the CountVectorizer we convert the text columns of our train, validation and  test data into three sparse matrices.

In [ ]:
bows_train = cv.fit_transform(df_train.text)
bows_train.sort_indices() # comes from TF2.0 sparse implementation, obscure requirement
bow_length = bows_train.shape[1]
print("BoW length:", bow_length)
bows_train

BoW length: 375


<5668x375 sparse matrix of type '<class 'numpy.int64'>'
	with 63159 stored elements in Compressed Sparse Row format>

In [ ]:
print(bows_train[0,:])

bows_valid = cv.transform(df_valid.text)
bows_valid.sort_indices() # comes from TF2.0 sparse implementation, obscure requirement
bows_test = cv.transform(df_test.text)
bows_test.sort_indices() # comes from TF2.0 sparse implementation, obscure requirement

  (0, 14)	1
  (0, 49)	1
  (0, 54)	2
  (0, 100)	1
  (0, 187)	1
  (0, 258)	1
  (0, 366)	1


# The model

We build a feed-forward neural network for our binary classification task, which will be trained with cross-entropy loss and minibatch SGD.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Parameters
############

hidden_size = 100

# Model
#######

inputs = Input(shape=(bow_length,))

# Hidden layer

hidden_output = Dense(hidden_size, activation='relu')(inputs)

# Softmax 

predictions = Dense(2, activation='softmax')(hidden_output)


# Full model

model = Model(inputs=inputs, outputs=predictions)

# Optimizer
####################

optimizer = SGD(lr=0.1)
 

# Compilation and fitting 
#########################

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy', # we use this cross entropy variant as the input is not 
              metrics=['accuracy'])

# Training

model.fit(x=bows_train, 
          y=df_train.sentiment.values,
          validation_data=(bows_valid, df_valid.sentiment.values),
          epochs=10,
          batch_size=200)

Train on 5668 samples, validate on 709 samples
Epoch 1/10
5668/5668 [==============================] - 0s 28us/sample - loss: 0.5351 - accuracy: 0.8296 - val_loss: 0.3995 - val_accuracy: 0.9295
Epoch 2/10
5668/5668 [==============================] - 0s 14us/sample - loss: 0.3191 - accuracy: 0.9446 - val_loss: 0.2514 - val_accuracy: 0.9661
Epoch 3/10
5668/5668 [==============================] - 0s 16us/sample - loss: 0.2056 - accuracy: 0.9642 - val_loss: 0.1739 - val_accuracy: 0.9718
Epoch 4/10
5668/5668 [==============================] - 0s 16us/sample - loss: 0.1471 - accuracy: 0.9704 - val_loss: 0.1325 - val_accuracy: 0.9788
Epoch 5/10
5668/5668 [==============================] - 0s 10us/sample - loss: 0.1146 - accuracy: 0.9734 - val_loss: 0.1077 - val_accuracy: 0.9803
Epoch 6/10
5668/5668 [==============================] - 0s 14us/sample - loss: 0.0945 - accuracy: 0.9760 - val_loss: 0.0919 - val_accuracy: 0.9803
Epoch 7/10
5668/5668 [==============================] - 0s 17us/sample 

In [ ]:
# Implement a function, that 
# assumes model and CountVectorizer to be available in the global namespace
# receives a string input
# processes it with CountVectorizer
# takes the processed BOW representation
# feeds it to the model for prediction
# formats the prediction as a dict with the keys "Negative" and "Positive"
# 0th prediction corresponds to Negative probability, 1st to Positive
# cast the probabilities to simple Python floats, Numpy floats do NOT work.
# return this dict
def predict_sentiment(input_string):
    global model
    global cv
    goodbad = ["Negative","Positive"]
    bow = cv.transform([input_string])
    prob_pred = model.predict(bow[0])
    return {"Negative":float(prob_pred[0,0]),"Positive":float(prob_pred[0,1])}

In [ ]:
# Test the function!
predict_sentiment("This film was pretty amazing.")

[[0.29418302 0.705817  ]]


{'Negative': 0.29418301582336426, 'Positive': 0.7058169841766357}

#Task1

In [ ]:
# Import Gradio, and build an interface
# Input is a textbox, outputs are "label"-s, and interpretation is set to default.

import gradio as gr
iface = gr.Interface(
  fn=predict_sentiment, inputs=gr.inputs.Textbox(default="This film was pretty amazing"),
  outputs="label", interpretation="default")

# Launch the interface, possibly use debug=True to make your life easier!
iface.launch()#debug=True)

Additional tasks: 

- Use the model interpretation tool of Gradio to observe some counterexamples, that do not work well!
- Set the flagging folder and use the flagging capability of Gradio to collect 15 badly behaving examples in a CSV file.

#Task2

In [ ]:
from urllib.request import urlretrieve
urlretrieve("https://gr-models.s3-us-west-2.amazonaws.com/mnist-model.h5", "mnist-model.h5")
model = tf.keras.models.load_model("mnist-model.h5")

In [ ]:
def recognize_digit(image):
    image = image.reshape(1, -1)
    prediction = model.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

In [ ]:
im = gr.inputs.Image(shape=(28, 28), image_mode='L', invert_colors=False, source="canvas")

In [ ]:
iface = gr.Interface(
    recognize_digit, 
    im, 
    gradio.outputs.Label(num_top_classes=3),
    live=True,
    interpretation="default",
    capture_session=True,
)

iface.launch()